In [1]:
import numpy as np
import cv2
import time
import sys

In [2]:
sys.path.append("./Funciones")
from funcion_segmenta_caracteres_matricula import funcion_segmenta_caracteres_matricula
from funcion_reconoce_caracteres_matricula import funcion_reconoce_caracteres_matricula
from funcion_representa_resultados import representa_resultados_segmentacion_reconocimiento
sys.path.remove("./Funciones")

In [3]:
%matplotlib tk

In [4]:
Ic_placa = cv2.imread('Material_Imagenes_Plantillas/01_Training/Training_01.jpg')

IsegEtiq, numObjetos, centroides, contornos = funcion_segmenta_caracteres_matricula(Ic_placa, True)

prediccion, angulos, datos = funcion_reconoce_caracteres_matricula(IsegEtiq, numObjetos, contornos)

representa_resultados_segmentacion_reconocimiento(Ic_placa, prediccion, centroides, contornos)

In [5]:
numImagenes = 25

Matriculas = [
    '7824BLX','H0504S','1374BXC','8959DDY','3189FYY',
    '4787DCX','H2305AB','H0853Z','H2462Y','H0612Y',
    '3189FYY','4787DCX','7226BLK','3680FSH','1675FLR',
    '9315FTC','2904CNN','8959DDY','8959DDY','H0612Y',
    '9014FCF','8585GBX','H0853Z','H2305AB','H2305AB'
]

Caracteres = '0123456789ABCDFGHKLNRSTXYZ'

In [6]:
tiempo = np.zeros(numImagenes)

numCaracteresTot = 0

numAciertos = 0
numAciertos_SinRotacion = 0

posiblesErrores = []
plantillasValidas = []
diferenciasCorrelacion = []
infoCaracter = []

for i in range(1, numImagenes+1):

    if i <= 20:
        rutaImagen = f'Material_Imagenes_Plantillas/02_Test/Test_{i:02d}.jpg'
    else:
        rutaImagen = f'Material_Imagenes_Plantillas/01_Training/Training_{i-20:02d}.jpg'

    start = time.time()

    Ic_placa = cv2.imread(rutaImagen)
    IsegEtiq, numObjetos, centroides, contornos = funcion_segmenta_caracteres_matricula(Ic_placa, False)
    prediccion, angulos, datos = funcion_reconoce_caracteres_matricula(IsegEtiq, numObjetos, contornos)

    end = time.time()

    tiempo[i-1] = end - start

    matriculaReal = Matriculas[i-1]

    for j in range(len(prediccion)):

        numCaracteresTot += 1
        
        if prediccion[j] == matriculaReal[j]:
            numAciertos += 1  

        datosCaracter = datos[:, :, j]
        datosCaracterSinRotacion = datosCaracter[:, 3]
        pos = np.argmax(datosCaracterSinRotacion)
        PrediccionSinRotacion = Caracteres[pos]

        if PrediccionSinRotacion == matriculaReal[j]:
            numAciertos_SinRotacion += 1
        else:
            posiblesErrores.append([i, prediccion[j], PrediccionSinRotacion])
            
        valoresOrd = np.sort(datosCaracter.flatten())[::-1]

        posCaracter = np.where(datosCaracter == valoresOrd[0])[0][0]

        for numPlant in range(1, len(valoresOrd)):
            c = np.where(datosCaracter == valoresOrd[numPlant])[0][0]
            if posCaracter != c:
                posCaracterSig = c
                break

        plantillasValidas.append(numPlant)
        diferenciasCorrelacion.append(valoresOrd[0] - valoresOrd[numPlant])

        infoCaracter.append([i, prediccion[j], Caracteres[posCaracterSig]])

In [7]:
print("TIEMPO COMPUTACIONAL")
print("-"*115)
for i in range(0, 5):
    print(f"Matricula {i+1}: {tiempo[i]:.2f}s", end ="\t")
    print(f"Matricula {i+6}: {tiempo[i+5]:.2f}s", end ="\t")
    print(f"Matricula {i+11}: {tiempo[i+10]:.2f}s", end ="\t")
    print(f"Matricula {i+16}: {tiempo[i+15]:.2f}s", end ="\t")
    print(f"Matricula {i+21}: {tiempo[i+20]:.2f}s")

print(f"\nTiempo promedio: {np.mean(tiempo):.4f}s")

print(f"\nTiempo total: {np.sum(tiempo):.4f}s")

TIEMPO COMPUTACIONAL
-------------------------------------------------------------------------------------------------------------------
Matricula 1: 0.13s	Matricula 6: 0.13s	Matricula 11: 0.11s	Matricula 16: 0.13s	Matricula 21: 0.10s
Matricula 2: 0.14s	Matricula 7: 0.15s	Matricula 12: 0.11s	Matricula 17: 0.12s	Matricula 22: 0.13s
Matricula 3: 0.11s	Matricula 8: 0.10s	Matricula 13: 0.11s	Matricula 18: 0.11s	Matricula 23: 0.10s
Matricula 4: 0.16s	Matricula 9: 0.10s	Matricula 14: 0.11s	Matricula 19: 0.12s	Matricula 24: 0.11s
Matricula 5: 0.15s	Matricula 10: 0.10s	Matricula 15: 0.12s	Matricula 20: 0.11s	Matricula 25: 0.13s

Tiempo promedio: 0.1194s

Tiempo total: 2.9851s


In [8]:
tasaAcierto = numAciertos / numCaracteresTot
tasaAciertoSinAngulos = numAciertos_SinRotacion / numCaracteresTot

print("TASAS DE ACIERTO")
print("----------------------------------------")
print(f"Tasa de acierto con rotaciones: {tasaAcierto*100:.2f} %")
print(f"Tasa de acierto sin rotaciones: {tasaAciertoSinAngulos*100:.2f} %")
print("")
print("POSIBLES ERRORES")
print("--------------------------------")
for i in range(len(posiblesErrores)):
    print("Matricula:", posiblesErrores[i][0])
    print("Valor predicho con rotaciones:", posiblesErrores[i][1])
    print("Valor predicho sin rotaciones:", posiblesErrores[i][2])
    print("")

TASAS DE ACIERTO
----------------------------------------
Tasa de acierto con rotaciones: 100.00 %
Tasa de acierto sin rotaciones: 98.22 %

POSIBLES ERRORES
--------------------------------
Matricula: 7
Valor predicho con rotaciones: B
Valor predicho sin rotaciones: 8

Matricula: 24
Valor predicho con rotaciones: B
Valor predicho sin rotaciones: 8

Matricula: 25
Valor predicho con rotaciones: B
Valor predicho sin rotaciones: 8



In [9]:
plantillasValidas = np.array(plantillasValidas)
diferenciasCorrelacion = np.array(diferenciasCorrelacion)

promedioPlantillas = np.mean(plantillasValidas)
desviacionPlantillas = np.std(plantillasValidas)
promedioDiferencias = np.mean(diferenciasCorrelacion)
desviacionDiferencias = np.std(diferenciasCorrelacion)

print("ROBUSTEZ")
print("--------------------------------------------------")
print(f"Promedio de plantillas válidas: {promedioPlantillas:.2f}")
print(f"Desviación de plantillas válidas: {desviacionPlantillas:.2f}")
print("")
print(f"Promedio de la diferencia de correlación: {promedioDiferencias:.4f}")
print(f"Desviación de la diferencia de correlación: {desviacionDiferencias:.4f}")

ROBUSTEZ
--------------------------------------------------
Promedio de plantillas válidas: 4.48
Desviación de plantillas válidas: 1.76

Promedio de la diferencia de correlación: 0.2391
Desviación de la diferencia de correlación: 0.1324


In [10]:
conflictos = []

for i in range(numCaracteresTot):
    conflictos.append([i, plantillasValidas[i], diferenciasCorrelacion[i]])

conflictos.sort(key=lambda x: (x[1], x[2]))

print("5 CASOS MÁS CONFLICTIVOS")
print("----------------------------------")
for i in range(5):
    idx = conflictos[i][0]
    print(f"CASO {i+1}:")
    print(f"Matricula: {infoCaracter[idx][0]}")
    print(f"Caracter predicho: {infoCaracter[idx][1]}")
    print(f"Plantillas validas del caracter: {conflictos[i][1]}")
    print(f"Diferencia de correlación: {conflictos[i][2]:.4f}")
    print(f"Siguiente caracter predicho: {infoCaracter[idx][2]}")
    print("")

5 CASOS MÁS CONFLICTIVOS
----------------------------------
CASO 1:
Matricula: 17
Caracter predicho: 0
Plantillas validas del caracter: 1
Diferencia de correlación: 0.0018
Siguiente caracter predicho: D

CASO 2:
Matricula: 2
Caracter predicho: 0
Plantillas validas del caracter: 1
Diferencia de correlación: 0.0175
Siguiente caracter predicho: D

CASO 3:
Matricula: 2
Caracter predicho: 0
Plantillas validas del caracter: 1
Diferencia de correlación: 0.0218
Siguiente caracter predicho: D

CASO 4:
Matricula: 8
Caracter predicho: 0
Plantillas validas del caracter: 1
Diferencia de correlación: 0.0245
Siguiente caracter predicho: D

CASO 5:
Matricula: 23
Caracter predicho: 0
Plantillas validas del caracter: 1
Diferencia de correlación: 0.0280
Siguiente caracter predicho: D

